In [ ]:
import os
from pathlib import Path
import sys

from dotenv import load_dotenv
import pandas as pd


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.domain.balances import get_balances_of_account
from src.domain.transactions import get_all_transactions
from src.infrastructure.google_sheets import update_google_sheet
from src.infrastructure.local_storage import get_all_acc_states_from_local_storage
from src.infrastructure.pdf_parser import (
    extract_pdf_lines,
    extract_text_from_pdf_in_local,
)

# Get files

In [ ]:
credentials_path = Path('../credentials/cool-plasma-452619-v4-feb20b70d461.json')
downloads_path = Path.home() / 'Downloads'

In [ ]:
files = get_all_acc_states_from_local_storage(downloads_path)
files

In [ ]:
for file in files:
    print(file)

In [ ]:
file_path = files[1]
full_pdf_text = extract_text_from_pdf_in_local(file_path)
lines = extract_pdf_lines(full_pdf_text)

In [ ]:
old_balance_index, new_balance_index = get_balances_of_account(lines)
all_transactions = get_all_transactions(lines, old_balance_index, new_balance_index)

# Google Connection

In [ ]:
from infrastructure.google_auth import get_google_clients


client, service = get_google_clients(credentials_path)
load_dotenv()
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID')

In [ ]:
spreadsheet = client.open_by_key(SPREADSHEET_ID)

sheet_transactions = spreadsheet.worksheet('Ausgaben')

df_transactions = pd.DataFrame(sheet_transactions.get_all_values())
df_transactions.columns = df_transactions.iloc[0]
df_transactions = df_transactions[1:].reset_index(drop=True)

In [ ]:
df_transactions.head()

In [ ]:
from domain.files import get_year_from_file_name


acc_state_year = get_year_from_file_name(all_transactions[0]['name'])

In [ ]:
from domain.transactions import append_transaction, extract_transaction_info


for transaction_index in range(len(all_transactions)):
    transaction = all_transactions[transaction_index]

    # Extract transaction info
    t_name, t_value, t_type, t_date = extract_transaction_info(
        transaction, acc_state_year
    )

    # Add new row to dataframe
    append_transaction(df_transactions, t_name, t_value, t_type, t_date)

# Load data into Google Sheet

In [ ]:
update_google_sheet(sheet_transactions, df_transactions)
print('✅ All changes saved to Google Sheets!')